In [306]:
# %pip install -q wordcloud

In [307]:
from collections import Counter

In [308]:
import nltk
from nltk import tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import wordcloud

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [309]:
# считываем книги
book1 = open('1981_crane_and_lightnings.txt').read()
book2 = open('1992_bronze_boy.txt').read()
book3 = open('1996_grandson_and_his_brothers.txt').read()
book4 = open('2004_venus_across_the_sun.txt').read()
book5 = open('2005_stomp_of_chess_horses.txt').read()
book6 = open('2010_topol_kids.txt').read()

books = [book1, book2, book3, book4, book5, book6]

In [310]:
# переписываем книги отдельными токенами
books_words = []
tok = nltk.tokenize.WhitespaceTokenizer()
for i in range(0, 6):
    books_words.append(tok.tokenize(books[i]))

In [311]:
# считаем токены и записываем их в переменную
tok = nltk.tokenize.WhitespaceTokenizer()
words_per_book = []
for i in range(0, 6):
    words_per_book.append(len(tok.tokenize(books[i])))
    words_per_book_print = len(tok.tokenize(books[i]))
    print(f'В {i+1} книге {words_per_book_print} слов')
words_per_book

В 1 книге 82756 слов
В 2 книге 88312 слов
В 3 книге 67238 слов
В 4 книге 11973 слов
В 5 книге 14879 слов
В 6 книге 81272 слов


[82756, 88312, 67238, 11973, 14879, 81272]

### `Лексическое разнообразие`

In [312]:
# считаем кол-во и проценты уникальных слов
unique_words_per_book = []
unique_words_percentage = []
for i in range(0, 6):
    unique_words_per_book_print = len(set(tok.tokenize(books[i])))
    print(f'В {i+1} книге {unique_words_per_book_print} уникальных слов')
    print(f'В процентах кол-во уникальных слов на текст составляет: {unique_words_per_book_print / words_per_book[i] * 100}%\n')
    unique_words_per_book.append(unique_words_per_book_print)
    unique_words_percentage.append(unique_words_per_book_print / words_per_book[i] * 100)
print(unique_words_per_book)
print(unique_words_percentage)

В 1 книге 25116 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 30.349461066267096%

В 2 книге 30234 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 34.235437992571796%

В 3 книге 22299 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 33.16428210238258%

В 4 книге 5686 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 47.49018625240124%

В 5 книге 6904 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 46.400967806976276%

В 6 книге 27648 уникальных слов
В процентах кол-во уникальных слов на текст составляет: 34.01909636775273%

[25116, 30234, 22299, 5686, 6904, 27648]
[30.349461066267096, 34.235437992571796, 33.16428210238258, 47.49018625240124, 46.400967806976276, 34.01909636775273]


может показаться, что в книгах 4 и 5 идет резкое возрастание языкового разнообразия, <br> если смотреть на процентное соотношение уникальных слов текста ко всему тексту. Так ли это?

In [313]:
df1 = pd.DataFrame(words_per_book, columns =['all_words'])
df2 = pd.DataFrame(unique_words_per_book, columns =['unique_words'])
df3 = pd.DataFrame(unique_words_percentage, columns =['percentage'])
data_words = pd.concat([df1, df2], axis=1)
data_percent = df3
display(data_words)

,all_words,unique_words
0,82756,25116
1,88312,30234
2,67238,22299
3,11973,5686
4,14879,6904
5,81272,27648


In [314]:
fig = df3.plot.bar()
fig.show()

Визуализация процентов наглядно показывает нам, что действительно есть скачок роста

In [316]:
df = pd.DataFrame(dict(data_words))
fig = df.plot.line()
fig.show()

Однако если посмотреть на ститистику по всем словам в книгах и уникальным, то можно заметить, <br> что именно в книгах где лексическое разнообразие наибольше, идет спад на количество слов в тексте в целом.<br> За счет этого уникальные слова и начинают составлять бОльшую долю текста.

In [317]:
data_all_percent = pd.concat([df1, df3], axis=1)
data_all_percent['all_words'] = data_all_percent['all_words'] / 1900

In [318]:
df = pd.DataFrame(dict(data_all_percent))
fig = df.plot.line()
fig.show()

Проверим это на графике взаимоотношения между количеством слов в книгах и процентным соотношением их к уникальным. <br> Для этого разделим количество слов на рандомный множитель для наглядности, но при этом не потеряв пропорциональное отношение. <br>Действительно, при резком спаде количества слов - идет резкий скачок процентов уникальных слов. <br> В результате чего можно заключить, что в целом лексическое разнообразие у Крапивина из года в года +- the same.<br> Не считая выбросов, с причиной которых мы разобрались.

### `Количество частей речи`

In [319]:
import spacy
from spacy import displacy

In [320]:
nlp = spacy.load('ru_core_news_sm')

In [321]:
doc1 = nlp(book1)
doc2 = nlp(book2)
doc3 = nlp(book3)
doc4 = nlp(book4)
doc5 = nlp(book5)
doc6 = nlp(book6)

In [322]:
docs = [doc1, doc2, doc3, doc4, doc5, doc6]

In [378]:
# for token in doc1:
#     list_token1.append(token.pos_)
counts1 = Counter([token.pos_ for token in doc1])
sorted_speech1 = {n: counts1[n] for n in sorted(counts1)}
df_sorted_speech1 = pd.DataFrame.from_dict(sorted_speech1, orient='index', columns = ['number1']).reset_index()

counts2 = Counter([token.pos_ for token in doc2])
sorted_speech2 = {n: counts2[n] for n in sorted(counts2)}
df_sorted_speech2 = pd.DataFrame.from_dict(sorted_speech2, orient='index', columns = ['number2']).reset_index()

counts3 = Counter([token.pos_ for token in doc3])
sorted_speech3 = {n: counts3[n] for n in sorted(counts3)}
df_sorted_speech3 = pd.DataFrame.from_dict(sorted_speech3, orient='index', columns = ['number3']).reset_index()

counts4 = Counter([token.pos_ for token in doc4])
sorted_speech4 = {n: counts4[n] for n in sorted(counts4)}
df_sorted_speech4 = pd.DataFrame.from_dict(sorted_speech4, orient='index', columns = ['number4']).reset_index()

counts5 = Counter([token.pos_ for token in doc5])
sorted_speech5 = {n: counts5[n] for n in sorted(counts5)}
df_sorted_speech5 = pd.DataFrame.from_dict(sorted_speech5, orient='index', columns = ['number5']).reset_index()

counts6 = Counter([token.pos_ for token in doc6])
sorted_speech6 = {n: counts6[n] for n in sorted(counts6)}
df_sorted_speech6 = pd.DataFrame.from_dict(sorted_speech6, orient='index', columns = ['number6']).reset_index()

data12 = df_sorted_speech1.merge(df_sorted_speech2, on='index')
data123 = data12.merge(df_sorted_speech3, on='index')
data1234 = data123.merge(df_sorted_speech4, on='index')
data12345 = data1234.merge(df_sorted_speech5, on='index')
dataspeech = data12345.merge(df_sorted_speech6, on='index')

# data_parts_speech = pd.concat([df_sorted_speech1, df_sorted_speech2, df_sorted_speech3, df_sorted_speech4, df_sorted_speech5, df_sorted_speech6], axis=1, join="inner")
# data_parts_speech

dataspeech['number1'] = dataspeech['number1'] / words_per_book[0] * 100
dataspeech['number2'] = dataspeech['number2'] / words_per_book[1] * 100
dataspeech['number3'] = dataspeech['number3'] / words_per_book[2] * 100
dataspeech['number4'] = dataspeech['number4'] / words_per_book[3] * 100
dataspeech['number5'] = dataspeech['number5'] / words_per_book[4] * 100
dataspeech['number6'] = dataspeech['number6'] / words_per_book[5] * 100

In [396]:
data_adj = dataspeech.query('index == "ADJ"')
data_adv = dataspeech.query('index == "ADV"')
data_verb = dataspeech.query('index == "VERB"')
data_noun = dataspeech.query('index == "NOUN"')
data_punct = dataspeech.query('index == "PUNCT"')


In [391]:
data_adj = data_adj.drop(['index'], axis=1)
fig1 = data_adj.T.plot.bar()
fig1.show()

В целом красноречивость по эпитетам у Крапивина одинакова от книги к книге, но в 2005 году в пятой книге, чуть больше разошелся на описания.

In [397]:
data_adv = data_adv.drop(['index'], axis=1)
fig2 = data_adv.T.plot.bar()
fig2.show()

Помимо прилагательных из прошлой таблички, не пожалел еще наречий для описаний в пятой книге. Рост наречий так же затронул и 4 книгу, написанную за год до пятой, рядом по времени написания.

In [398]:
data_noun = data_noun.drop(['index'], axis=1)
fig3 = data_noun.T.plot.bar()
fig3.show()

В целом, уровень существительных стабилен от книги к книге.

In [399]:
data_punct = data_punct.drop(['index'], axis=1)
fig4 = data_punct.T.plot.bar()
fig4.show()

Можно заметить по оси y, что процентное соотношение знаков препинания к словам в тексте достаточно высокое в каждой книге, аж до 35% бывает. А всё по тому, что Крапивин часто использует диалоги в своих произведениях, которые вводятся с знака "--"

### `Динамика текста`

In [400]:
data_verb = data_verb.drop(['index'], axis=1)
fig5 = data_verb.T.plot.bar()
fig5.show()

оцениваем динамику текста через динамичные глаголы. Особеено высоко процентное отношение глаголов к тексту в в первой книге 1981 года, в начале авторского пути.

### `Word Cloud`

In [325]:
# import spacy
# nlp = spacy.load('ru_core_news_sm')
# doc = nlp(book_1_words)

In [411]:
tok = nltk.tokenize.WhitespaceTokenizer()
book1_words = str(tok.tokenize(book1))
book2_words = str(tok.tokenize(book2))
book3_words = str(tok.tokenize(book3))
book4_words = str(tok.tokenize(book4))
book5_words = str(tok.tokenize(book5))
book6_words = str(tok.tokenize(book6))

In [412]:
doc11 = nlp(book1_words)
doc22 = nlp(book2_words)
doc33 = nlp(book3_words)
doc44 = nlp(book4_words)
doc55 = nlp(book5_words)
doc66 = nlp(book6_words)

In [ ]:
# stop = stopwords.words('russian')
# clean_words = [word for word in doc1 if word.lower() not in stop]

In [ ]:
# wordcloud = WordCloud().generate(doc1)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()